In [1]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [2]:
dataset = 'DB2022'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/pr2c00244_si_005.csv', header=5)
print('Raw data loaded.')
data

Loading raw data for DB2022 ...
Raw data loaded.


,Region,Uniprot ID,Gene Name,Type,Peptide,Phospho (STY) Probabilities,Positions within proteins,Position in peptide,Charge,LFQ intensity L1,...,Amino Acid,Intensity L1,Intensity L2,Intensity R1,Intensity R2,Unnamed: 42,Unnamed: 43,Type III,The proteins having LFQ intensities present in RH only in phospho(STY) but LFQ intensities present in both LH and RH in protein group files,Unnamed: 46
0,Amygdala,O00499,BIN1,Type I,GNKSPSPPDGSPAATPEIR,GNKS(1)PS(1)PPDGSPAATPEIR,298,6.0,2.0,17217000.0,...,S,118000000.0,94519000.0,29562000.0,22336000.0,NaN,NaN,NaN,NaN,NaN
1,Amygdala,O00499,BIN1,Type II,GNKSPSPPDGSPAATPEIR,GNKS(1)PS(1)PPDGSPAATPEIR,296,4.0,2.0,17217000.0,...,S,122000000.0,111000000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,Amygdala,O14974,PPP1R12A,Type III,RSTQGVTLTDLQEAEK,RS(0.237)T(0.757)QGVT(0.006)LT(0.001)DLQEAEK,696,3.0,3.0,196540000.0,...,T,0.0,0.0,2540400.0,2767100.0,NaN,NaN,NaN,NaN,NaN
3,Amygdala,O15075,DCLK1,Type I,SPSPSPTSPGSLRK,SPSPSPT(0.053)S(0.894)PGS(0.053)LRK,337,8.0,2.0,26523000.0,...,S,34292000.0,27663000.0,20336000.0,14994000.0,NaN,NaN,NaN,NaN,NaN
4,Amygdala,O43301,HSPA12A,Type III,TPTTILLTPERK,TPTTILLT(1)PERK,106,8.0,2.0,95646000.0,...,T,0.0,0.0,1788700.0,1815000.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q14194,CRMP1
3394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q7L0J3,SV2A
3395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q8WXG6,MADD
3396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q9UPA5,BSN


In [3]:
# filter data to keep only those with localization probability >= 0.85
data = data[data['Localization prob'] >= 0.85] 

# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino Acid'].str.contains(';', na=False)]
data = data[~data['Positions within proteins'].str.contains(';', na=False)]
data = data[~data['Gene Name'].str.contains(';', na=False)]
data

,Region,Uniprot ID,Gene Name,Type,Peptide,Phospho (STY) Probabilities,Positions within proteins,Position in peptide,Charge,LFQ intensity L1,...,Amino Acid,Intensity L1,Intensity L2,Intensity R1,Intensity R2,Unnamed: 42,Unnamed: 43,Type III,The proteins having LFQ intensities present in RH only in phospho(STY) but LFQ intensities present in both LH and RH in protein group files,Unnamed: 46
0,Amygdala,O00499,BIN1,Type I,GNKSPSPPDGSPAATPEIR,GNKS(1)PS(1)PPDGSPAATPEIR,298,6.0,2.0,17217000.0,...,S,118000000.0,94519000.0,29562000.0,22336000.0,NaN,NaN,NaN,NaN,NaN
1,Amygdala,O00499,BIN1,Type II,GNKSPSPPDGSPAATPEIR,GNKS(1)PS(1)PPDGSPAATPEIR,296,4.0,2.0,17217000.0,...,S,122000000.0,111000000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,Amygdala,O15075,DCLK1,Type I,SPSPSPTSPGSLRK,SPSPSPT(0.053)S(0.894)PGS(0.053)LRK,337,8.0,2.0,26523000.0,...,S,34292000.0,27663000.0,20336000.0,14994000.0,NaN,NaN,NaN,NaN,NaN
4,Amygdala,O43301,HSPA12A,Type III,TPTTILLTPERK,TPTTILLT(1)PERK,106,8.0,2.0,95646000.0,...,T,0.0,0.0,1788700.0,1815000.0,NaN,NaN,NaN,NaN,NaN
5,Amygdala,O43301,HSPA12A,Type III,SPLTYGVGVLNR,S(0.999)PLT(0.001)YGVGVLNR,533,1.0,2.0,95646000.0,...,S,0.0,0.0,2431500.0,2014400.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3311,Hippocampus,Q9H4G0,EPB41L1,Type III,AVVYRETDPSPEER,AVVY(0.004)RET(0.015)DPS(0.981)PEER,870,10.0,3.0,17747000.0,...,S,0.0,0.0,4942500.0,5703000.0,NaN,NaN,NaN,NaN,NaN
3312,Hippocampus,Q9UN36,NDRG2,Type III,TASLTSAASVDGNR,TASLT(0.003)S(0.006)AAS(0.991)VDGNR,338,9.0,2.0,55890000.0,...,S,0.0,0.0,3785800.0,4031000.0,NaN,NaN,NaN,From,To
3314,Hippocampus,Q9UQM7,CAMK2A,Type III,SDGVKESSESTNTTIEDEDTKVR,S(0.001)DGVKES(0.026)S(0.022)ES(0.068)T(0.153)...,336,13.0,3.0,87831000.0,...,T,0.0,0.0,101640000.0,26955000.0,NaN,NaN,NaN,O00499,BIN1
3315,Hippocampus,Q9UQM7,CAMK2A,Type III,SDGVKESSESTNTTIEDEDTKVR,S(0.001)DGVKES(0.026)S(0.022)ES(0.068)T(0.153)...,337,14.0,3.0,87831000.0,...,T,0.0,0.0,76505000.0,26955000.0,NaN,NaN,NaN,O75379,VAMP4


In [4]:
# filter data
data['Peptide'] = data['Peptide'].str.replace('_', '')
data

,Region,Uniprot ID,Gene Name,Type,Peptide,Phospho (STY) Probabilities,Positions within proteins,Position in peptide,Charge,LFQ intensity L1,...,Amino Acid,Intensity L1,Intensity L2,Intensity R1,Intensity R2,Unnamed: 42,Unnamed: 43,Type III,The proteins having LFQ intensities present in RH only in phospho(STY) but LFQ intensities present in both LH and RH in protein group files,Unnamed: 46
0,Amygdala,O00499,BIN1,Type I,GNKSPSPPDGSPAATPEIR,GNKS(1)PS(1)PPDGSPAATPEIR,298,6.0,2.0,17217000.0,...,S,118000000.0,94519000.0,29562000.0,22336000.0,NaN,NaN,NaN,NaN,NaN
1,Amygdala,O00499,BIN1,Type II,GNKSPSPPDGSPAATPEIR,GNKS(1)PS(1)PPDGSPAATPEIR,296,4.0,2.0,17217000.0,...,S,122000000.0,111000000.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,Amygdala,O15075,DCLK1,Type I,SPSPSPTSPGSLRK,SPSPSPT(0.053)S(0.894)PGS(0.053)LRK,337,8.0,2.0,26523000.0,...,S,34292000.0,27663000.0,20336000.0,14994000.0,NaN,NaN,NaN,NaN,NaN
4,Amygdala,O43301,HSPA12A,Type III,TPTTILLTPERK,TPTTILLT(1)PERK,106,8.0,2.0,95646000.0,...,T,0.0,0.0,1788700.0,1815000.0,NaN,NaN,NaN,NaN,NaN
5,Amygdala,O43301,HSPA12A,Type III,SPLTYGVGVLNR,S(0.999)PLT(0.001)YGVGVLNR,533,1.0,2.0,95646000.0,...,S,0.0,0.0,2431500.0,2014400.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3311,Hippocampus,Q9H4G0,EPB41L1,Type III,AVVYRETDPSPEER,AVVY(0.004)RET(0.015)DPS(0.981)PEER,870,10.0,3.0,17747000.0,...,S,0.0,0.0,4942500.0,5703000.0,NaN,NaN,NaN,NaN,NaN
3312,Hippocampus,Q9UN36,NDRG2,Type III,TASLTSAASVDGNR,TASLT(0.003)S(0.006)AAS(0.991)VDGNR,338,9.0,2.0,55890000.0,...,S,0.0,0.0,3785800.0,4031000.0,NaN,NaN,NaN,From,To
3314,Hippocampus,Q9UQM7,CAMK2A,Type III,SDGVKESSESTNTTIEDEDTKVR,S(0.001)DGVKES(0.026)S(0.022)ES(0.068)T(0.153)...,336,13.0,3.0,87831000.0,...,T,0.0,0.0,101640000.0,26955000.0,NaN,NaN,NaN,O00499,BIN1
3315,Hippocampus,Q9UQM7,CAMK2A,Type III,SDGVKESSESTNTTIEDEDTKVR,S(0.001)DGVKES(0.026)S(0.022)ES(0.068)T(0.153)...,337,14.0,3.0,87831000.0,...,T,0.0,0.0,76505000.0,26955000.0,NaN,NaN,NaN,O75379,VAMP4


In [5]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 

In [6]:
data = match_seq_to_genename(data, 'Peptide')

GNKSPSPPDGSPAATPEIR
Match found for sequence: ID: sp|O00499|BIN1_HUMAN
Name: sp|O00499|BIN1_HUMAN
Description: sp|O00499|BIN1_HUMAN Myc box-dependent-interacting protein 1 OS=Homo sapiens OX=9606 GN=BIN1 PE=1 SV=1
Number of features: 0
Seq('MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQCVQNFNKQ...RVP')
Gene name match: <re.Match object; span=(85, 92), match='GN=BIN1'>
Match found: GNKSPSPPDGSPAATPEIR -> BIN1
GNKSPSPPDGSPAATPEIR
Match found for sequence: ID: sp|O00499|BIN1_HUMAN
Name: sp|O00499|BIN1_HUMAN
Description: sp|O00499|BIN1_HUMAN Myc box-dependent-interacting protein 1 OS=Homo sapiens OX=9606 GN=BIN1 PE=1 SV=1
Number of features: 0
Seq('MAEMGSKGVTAGKIASNVQKKLTRAQEKVLQKLGKADETKDEQFEQCVQNFNKQ...RVP')
Gene name match: <re.Match object; span=(85, 92), match='GN=BIN1'>
Match found: GNKSPSPPDGSPAATPEIR -> BIN1
SPSPSPTSPGSLRK
Match found for sequence: ID: sp|O15075|DCLK1_HUMAN
Name: sp|O15075|DCLK1_HUMAN
Description: sp|O15075|DCLK1_HUMAN Serine/threonine-protein kinase DCLK1 OS=Homo 

In [7]:
data['Phosphosite'] = data['Amino Acid'].astype(str) + '(' + data['Positions within proteins'].astype(str) + ')'


In [8]:
# Keep only 'Phosphosite' and ratio columns
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Intensity' in col or 'LFQ intensity' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,LFQ intensity L1,LFQ intensity L2,LFQ intensity R1,LFQ intensity R2,LFQ intensity L1.1,LFQ intensity L2.1,LFQ intensity R1.1,LFQ intensity R2.1,LFQ intensity L1.2,LFQ intensity L2.2,LFQ intensity R1.2,LFQ intensity R2.2,Intensity L1,Intensity L2,Intensity R1,Intensity R2
0,S(298),BIN1,17217000.0,34115000.0,39132000.0,62882000.0,1.653288,0.262559,1.204876,1.352445,32079000.0,132080000.0,16969000.0,16774000.0,118000000.0,94519000.0,29562000.0,22336000.0
1,S(296),BIN1,17217000.0,34115000.0,39132000.0,62882000.0,1.653288,0.262559,1.204876,1.352445,32079000.0,132080000.0,16969000.0,16774000.0,122000000.0,111000000.0,0.0,0.0
3,S(337),DCLK1,26523000.0,62163000.0,10937000.0,26160000.0,-3.112991,-2.727027,-1.352219,-1.067588,41141000.0,54215000.0,38346000.0,14994000.0,34292000.0,27663000.0,20336000.0,14994000.0
4,T(106),HSPA12A,95646000.0,70886000.0,12814000.0,10965000.0,1.340219,1.647576,1.254526,0.180335,0.0,0.0,1788700.0,1815000.0,0.0,0.0,1788700.0,1815000.0
5,S(533),HSPA12B,95646000.0,70886000.0,12814000.0,10965000.0,1.340219,1.647576,1.254526,0.180335,0.0,0.0,2431500.0,2014400.0,0.0,0.0,2431500.0,2014400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3311,S(870),EPB41L1,17747000.0,20687000.0,53952000.0,50637000.0,-1.169682,-1.152414,-0.371283,-0.242912,0.0,0.0,4942500.0,5703000.0,0.0,0.0,4942500.0,5703000.0
3312,S(338),NDRG2,55890000.0,52218000.0,32636000.0,32171000.0,0.485332,0.183409,-1.096495,-0.897345,7887600.0,7831600.0,63192000.0,56947000.0,0.0,0.0,3785800.0,4031000.0
3314,T(336),CAMK2A,87831000.0,88073000.0,71405000.0,57596000.0,1.137474,0.937561,0.033066,-0.057137,33060000.0,6667200.0,190180000.0,181960000.0,0.0,0.0,101640000.0,26955000.0
3315,T(337),CAMK2A,87831000.0,88073000.0,71405000.0,57596000.0,1.137474,0.937561,0.033066,-0.057137,33060000.0,6667200.0,190180000.0,181960000.0,0.0,0.0,76505000.0,26955000.0


In [9]:
Intensity_columns = [col for col in data.columns if 'Intensity' in col or 'LFQ intensity' in col]
data[Intensity_columns] = data[Intensity_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_11745/1004942559.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Intensity_columns] = data[Intensity_columns].apply(pd.to_numeric, errors='coerce')


In [10]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.Dataframe>
    
    out:
    ====
    dataset: <pd.Dataframe> with log2 transformed values

    '''
    cols_to_transform = dataset[Intensity_columns]
    dataset[Intensity_columns] = cols_to_transform.apply(np.log2)
    print('Data has been log2 transformed.')
    return dataset


In [11]:
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}") # Print the DataFrame after log2 transformation
data

Data has been log2 transformed.
DataFrame after log2 transformation:
     Phosphosite GeneName  LFQ intensity L1  LFQ intensity L2  \
0         S(298)     BIN1         24.037330         25.023903   
1         S(296)     BIN1         24.037330         25.023903   
3         S(337)    DCLK1         24.660741         25.889553   
4         T(106)  HSPA12A         26.511201         26.078997   
5         S(533)  HSPA12B         26.511201         26.078997   
...          ...      ...               ...               ...   
3311      S(870)  EPB41L1         24.081072         24.302221   
3312      S(338)    NDRG2         25.736087         25.638044   
3314      T(336)   CAMK2A         26.388227         26.392196   
3315      T(337)   CAMK2A         26.388227         26.392196   
3316      S(443)  EPB41L3         23.895505         23.780567   

      LFQ intensity R1  LFQ intensity R2  LFQ intensity L1.1  \
0            25.221846         25.906144            0.725338   
1            25.221846

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_11745/2333207401.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[Intensity_columns] = cols_to_transform.apply(np.log2)


,Phosphosite,GeneName,LFQ intensity L1,LFQ intensity L2,LFQ intensity R1,LFQ intensity R2,LFQ intensity L1.1,LFQ intensity L2.1,LFQ intensity R1.1,LFQ intensity R2.1,LFQ intensity L1.2,LFQ intensity L2.2,LFQ intensity R1.2,LFQ intensity R2.2,Intensity L1,Intensity L2,Intensity R1,Intensity R2
0,S(298),BIN1,24.037330,25.023903,25.221846,25.906144,0.725338,-1.929287,0.268885,0.435570,24.935126,26.976837,24.016398,23.999723,26.814212,26.494101,24.817241,24.412868
1,S(296),BIN1,24.037330,25.023903,25.221846,25.906144,0.725338,-1.929287,0.268885,0.435570,24.935126,26.976837,24.016398,23.999723,26.862306,26.725984,-inf,-inf
3,S(337),DCLK1,24.660741,25.889553,23.382714,24.640859,NaN,NaN,NaN,NaN,25.294074,25.692189,25.192573,23.837882,25.031369,24.721454,24.277533,23.837882
4,T(106),HSPA12A,26.511201,26.078997,23.611218,23.386402,0.422469,0.720345,0.327143,-2.471248,-inf,-inf,20.770480,20.791538,-inf,-inf,20.770480,20.791538
5,S(533),HSPA12B,26.511201,26.078997,23.611218,23.386402,0.422469,0.720345,0.327143,-2.471248,-inf,-inf,21.213415,20.941919,-inf,-inf,21.213415,20.941919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3311,S(870),EPB41L1,24.081072,24.302221,25.685173,25.593689,NaN,NaN,NaN,NaN,-inf,-inf,22.236810,22.443290,-inf,-inf,22.236810,22.443290
3312,S(338),NDRG2,25.736087,25.638044,24.959961,24.939257,-1.042955,-2.446866,NaN,NaN,22.911155,22.900876,25.913239,25.763117,-inf,-inf,21.852167,21.942706
3314,T(336),CAMK2A,26.388227,26.392196,26.089522,25.779465,0.185834,-0.093015,-4.918517,NaN,24.978583,22.668650,27.502790,27.439046,-inf,-inf,26.598893,24.684050
3315,T(337),CAMK2A,26.388227,26.392196,26.089522,25.779465,0.185834,-0.093015,-4.918517,NaN,24.978583,22.668650,27.502790,27.439046,-inf,-inf,26.189051,24.684050


In [12]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_11745/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [13]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains(';', case=False, na=False)]
    data = data[~data.phosphosite_ID.str.contains('-', case=False, na=False)]

    # Add this line to remove decimals from phosphosite_ID (e.g., S123.0 -> S123)
    data['phosphosite_ID'] = data['phosphosite_ID'].apply(lambda x: re.sub(r'\((\d+)\.0+\)', r'(\1)', x))
    
    data_grouped = data.groupby(by='phosphosite_ID')
    
    if len(data) != len(data_grouped):
        numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
        non_numeric_cols = data.columns.difference(numeric_cols + ['phosphosite_ID']).tolist()
        data_numeric = data_grouped[numeric_cols].mean()
        data_categorical = data_grouped[non_numeric_cols].first().reset_index()
        
        # Merge numeric and non-numeric parts
        data = pd.merge(data_categorical, data_numeric, on='phosphosite_ID')
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')

    # Replace inf values with NaNs
    data = data.replace([np.inf, -np.inf], np.nan)
    
    # Ensure phosphosite_ID is first column
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)

    return data


In [14]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
After cleaning phosphosite_ID column:


,phosphosite_ID,LFQ intensity L1,LFQ intensity L2,LFQ intensity R1,LFQ intensity R2,LFQ intensity L1.1,LFQ intensity L2.1,LFQ intensity R1.1,LFQ intensity R2.1,LFQ intensity L1.2,LFQ intensity L2.2,LFQ intensity R1.2,LFQ intensity R2.2,Intensity L1,Intensity L2,Intensity R1,Intensity R2
0,AAK1_S(624),23.383920,23.391158,23.187633,22.926047,-1.905495,-1.162551,-0.616768,0.189059,21.635058,21.331015,21.254886,21.495691,22.471323,22.583672,22.183867,21.975911
1,AAK1_T(606),23.372388,23.160218,23.979369,23.302570,-1.914849,NaN,-1.143520,NaN,28.404300,28.356323,NaN,29.031916,NaN,20.072216,21.238881,NaN
2,AAK1_T(620),23.291190,23.277340,23.090021,23.095509,-0.755880,-0.364099,-0.696299,-1.125267,23.824318,22.960310,22.988973,22.638240,22.840506,22.725597,22.401598,22.423069
3,AAK1_T(673),23.261029,23.217866,23.471721,23.412019,-0.621036,0.120305,-0.999208,-2.392749,22.266363,21.403960,NaN,23.331870,NaN,NaN,NaN,23.380282
4,ABI1_S(183),26.219632,23.316171,21.991996,22.243827,NaN,NaN,NaN,NaN,NaN,NaN,24.596904,24.465568,NaN,NaN,22.975246,22.593067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,VPS35_S(7),25.656306,25.617800,23.980962,23.752015,0.554162,0.753988,0.682661,0.397223,25.628035,25.664642,24.227133,23.862247,25.628035,25.664642,24.227133,23.862247
919,WDR1_S(322),21.803418,21.817708,20.587172,20.530279,NaN,NaN,-2.947840,-1.670599,21.759754,21.783835,20.801452,20.754461,21.759754,21.783835,20.801452,20.754461
920,WDR1_S(392),21.312797,21.433288,18.675523,18.529934,NaN,NaN,NaN,NaN,19.243723,19.209363,NaN,NaN,19.243723,19.209363,NaN,NaN
921,WNK2_S(560),20.574132,20.573392,19.763105,19.636317,NaN,NaN,NaN,NaN,20.574132,20.573392,19.763105,19.636317,20.574132,20.573392,19.763105,19.636317


In [15]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/DB2022.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

DB2022 has been saved to CSV successfully!     phosphosite_ID  LFQ intensity L1  LFQ intensity L2  LFQ intensity R1  \
0      AAK1_S(624)         23.383920         23.391158         23.187633   
1      AAK1_T(606)         23.372388         23.160218         23.979369   
2      AAK1_T(620)         23.291190         23.277340         23.090021   
3      AAK1_T(673)         23.261029         23.217866         23.471721   
4      ABI1_S(183)         26.219632         23.316171         21.991996   
..             ...               ...               ...               ...   
918     VPS35_S(7)         25.656306         25.617800         23.980962   
919    WDR1_S(322)         21.803418         21.817708         20.587172   
920    WDR1_S(392)         21.312797         21.433288         18.675523   
921    WNK2_S(560)         20.574132         20.573392         19.763105   
922     ZAN_S(734)         28.075533         29.979065         27.106988   

     LFQ intensity R2  LFQ intensity L1.1  L